In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 
import cv2
import os
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from IPython.display import display
from LaneFinder import *

In [2]:
IMAGE_SIZE=(720, 1280)
CHESSBOARD_IMAGE_DIR = "camera_cal"
TEST_IMAGE_DIR = "test_images"
INPUT_VIDEO_DIR= "input_videos"
OUTPUT_IMAGE_DIR = "output_images"
OUTPUT_VIDEO_DIR = "output_videos"
ABSOLUTE_SOBEL_X = (7, 20, 100)
ABSOLUTE_SOBEL_Y = (7, 20, 100)
MAGNITUDE_SOBEL = (3, 30, 100)
DIRECTION_SOBEL = (31, 0.5, 1.0)
S_CHANNEL_THRESHOLD = (170, 255)
WARP_SRC = np.float32([(532, 496),
                       (756, 496),
                       (288, 664),
                       (1016, 664)])
WARP_DST = np.float32([(WARP_SRC[2][0], WARP_SRC[2][1] - 200),
                       (WARP_SRC[3][0], WARP_SRC[3][1] - 200),
                       (WARP_SRC[2][0], WARP_SRC[2][1]),
                       (WARP_SRC[3][0], WARP_SRC[3][1])])
SLIDING_WINDOW_PARAMS = (9, 100, 50)
METERS_PER_PIXEL_Y = 30 / 720
METERS_PER_PIXEL_X = 3.7 / 700
MAX_RECENT_XFITTED = 10
LANE_DETECTION_FAILURE_COUNT_BEFORE_SLIDING_WINDOW=20
REGION_OF_INTEREST_VERTS = np.array([[
            (0, IMAGE_SIZE[0]),
            (IMAGE_SIZE[1] / 2, IMAGE_SIZE[0] / 2 + 45),
            (IMAGE_SIZE[1] / 2, IMAGE_SIZE[0] / 2 + 45),
            (IMAGE_SIZE[1],     IMAGE_SIZE[0])
        ]], dtype = np.int32)

In [3]:
# initialize class instance containing advanced lane line detection methods
LaneFinder = LaneFinder(
    image_size=IMAGE_SIZE,
    chessboard_image_dir=CHESSBOARD_IMAGE_DIR,
    absolute_sobel_x=ABSOLUTE_SOBEL_X,
    absolute_sobel_y=ABSOLUTE_SOBEL_Y,
    magnitude_sobel=MAGNITUDE_SOBEL,
    direction_sobel=DIRECTION_SOBEL,
    s_channel_thresh=S_CHANNEL_THRESHOLD,
    warp_perspective=(WARP_SRC, WARP_DST),
    sliding_window_params=SLIDING_WINDOW_PARAMS,
    meters_per_pixel=(METERS_PER_PIXEL_Y, METERS_PER_PIXEL_X),
    max_recent_xfitted=MAX_RECENT_XFITTED,
    lane_detection_failure_count_before_sliding_window=LANE_DETECTION_FAILURE_COUNT_BEFORE_SLIDING_WINDOW,
    region_of_interest_verts=REGION_OF_INTEREST_VERTS,
)

In [4]:
def __main__():
    print("Starting the Pipeline for the Advanced Lane Finding...")
    print("First, test the pipeline for the images...")
    images()
    print("Done with The Images...")
    print("Starting the Pipeline for the Videos...")
    Videos()
    print("Done with Videos...")
    
def images():
    test_img_paths = os.listdir(TEST_IMAGE_DIR)
    for fname in test_img_paths:
        # the following hack needed to start sliding window search for each new image
        LaneFinder._left_line.detected = False
        LaneFinder._right_line.detected = False

        img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
        img_text = LaneFinder.pipeline(image=img)

        # create output directory for images, if does not exist
        if not os.path.isdir(OUTPUT_IMAGE_DIR):
            os.mkdir(OUTPUT_IMAGE_DIR)

        # save undistorted image
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_text.jpg"), img_text)

def Videos():
    test_video_paths = os.listdir(INPUT_VIDEO_DIR)
    
    # create output directory for vidoes, if does not exist
    if not os.path.isdir(OUTPUT_VIDEO_DIR):
            os.mkdir(OUTPUT_VIDEO_DIR)
            
    for fname in test_video_paths:
        # define paths to source and destination videos
        vid_src = os.path.join(INPUT_VIDEO_DIR, fname)
        vid_dst = os.path.join(OUTPUT_VIDEO_DIR, fname)
        print("Starting the "+ fname +" Video...")

        video = VideoFileClip(vid_src)
        video_clip = video.fl_image(LaneFinder.pipeline)
        %time video_clip.write_videofile(vid_dst, audio = False)
        display(HTML(
                """
                    <video width="960" height="540" controls>
                       <source src="{0}">
                    </video>
                """.format(vid_dst)))

In [5]:
__main__()

Starting the Pipeline for the Advanced Lane Finding...
First, test the pipeline for the images...
Done with The Images...
Starting the Pipeline for the Videos...
Starting the challenge_video.mp4 Video...
Moviepy - Building video output_videos\challenge_video.mp4.
Moviepy - Writing video output_videos\challenge_video.mp4



Moviepy - Done !
Moviepy - video ready output_videos\challenge_video.mp4
Wall time: 4min 57s


Starting the harder_challenge_video.mp4 Video...
Moviepy - Building video output_videos\harder_challenge_video.mp4.
Moviepy - Writing video output_videos\harder_challenge_video.mp4



Moviepy - Done !
Moviepy - video ready output_videos\harder_challenge_video.mp4
Wall time: 11min 47s


Starting the project_video.mp4 Video...


OSError: [WinError 6] The handle is invalid